In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import datetime
import re
import numpy as np
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import seaborn as sns
import numpy as np
from itertools import groupby
!pip install folium

## Read Data

In [2]:
orig_exonerations_df = pd.read_csv(r"C:\Users\cindy\OneDrive\Documents\DA15\Python\Projects\exonerations\data\nre_export_2025-07-24.csv")

In [3]:
orig_exonerations_df.head()

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
0,"Abbitt, Joseph",North Carolina,Forsyth,31.0,Male,Black,Child Sex Abuse,Life,NaN,14.2,...,No,No,No,No,No,No,No,No,No,NaN
1,"Abbott, Cinque",Illinois,Cook,19.0,Male,Black,Drug Possession or Sale,Probation,NaN,0.0,...,No,Yes,Yes,No,Yes,No,No,No,No,NaN
2,"Abbott, Stearns Kendall",Massachusetts,Middlesex,40.0,Male,White,Murder,Death,NaN,30.5,...,No,No,No,No,No,No,No,No,No,NaN
3,"Abdal, Warith Habib",New York,Erie,43.0,Male,Black,Sexual Assault,Term of Years,20 to Life,16.2,...,No,Yes,Yes,No,Yes,No,No,Yes,No,NaN
4,"Abernathy, Christopher",Illinois,Cook,17.0,Male,White,Murder,Life without parole,NaN,28.1,...,Yes,Yes,Yes,No,Yes,No,No,No,Yes,2006


## Cleaning

In [4]:
# Filling in empty cells with "Not Provided"
exonerations_df = orig_exonerations_df.fillna('Not Provided')

In [30]:
exonerations_df.columns

Index(['Name', 'State', 'County', 'Age at Crime Yrs', 'Sex', 'Race', 'Crime',
       'Sentence', 'Term of Years', 'Years Lost', 'Pre-1989', 'Date of Exon',
       'Date of 1st Convic', 'Date of Crime Month', 'Date of Crime Day',
       'Date of Crime Year ', 'Date of Release', 'Posting Date',
       'Case Characteristics', 'DNA', 'False Confession ',
       'False or Misleading Forensic Evidence?', 'Inadequate Legal Defense ',
       'Mistaken Witness ID', 'Official Misconduct',
       'Perjury or False Accusation?', 'Child Welfare Worker Misconduct',
       'Exculpatory Evidence Withheld', 'Forensic Analyst Misconduct',
       'Knowingly Permitting Perjury', 'Misconduct in Interrogation',
       'Misconduct in Interrogation.1', 'Misconduct that is not withholding',
       'OM - Perjury', 'Police Misconduct', 'Prosecutor Lied in Court',
       'Prosecutor Misconduct', 'Witness Tampering', 'Recantation',
       'Year of recantation'],
      dtype='object')

In [5]:
exonerations_df.head()

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
0,"Abbitt, Joseph",North Carolina,Forsyth,31.0,Male,Black,Child Sex Abuse,Life,Not Provided,14.2,...,No,No,No,No,No,No,No,No,No,Not Provided
1,"Abbott, Cinque",Illinois,Cook,19.0,Male,Black,Drug Possession or Sale,Probation,Not Provided,0.0,...,No,Yes,Yes,No,Yes,No,No,No,No,Not Provided
2,"Abbott, Stearns Kendall",Massachusetts,Middlesex,40.0,Male,White,Murder,Death,Not Provided,30.5,...,No,No,No,No,No,No,No,No,No,Not Provided
3,"Abdal, Warith Habib",New York,Erie,43.0,Male,Black,Sexual Assault,Term of Years,20 to Life,16.2,...,No,Yes,Yes,No,Yes,No,No,Yes,No,Not Provided
4,"Abernathy, Christopher",Illinois,Cook,17.0,Male,White,Murder,Life without parole,Not Provided,28.1,...,Yes,Yes,Yes,No,Yes,No,No,No,Yes,2006


In [6]:
exonerations_df['Race'] = exonerations_df['Race'].replace("Don't Know", "Unknown")

In [7]:
exonerations_df = exonerations_df.rename(columns={'Years lost': 'Years Lost'})

In [32]:
exonerations_df = exonerations_df.rename(columns={'Inadequate Legal Defense ': 'Inadequate Legal Defense'})

## Exploring Data

In [8]:
# Organizing by State
exonerations_df.sort_values(by = 'State')

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
3322,"Sealie, Frank",Alabama,Jefferson,23.0,Male,Black,Murder,Life without parole,Not Provided,0.8,...,No,No,No,No,No,No,No,No,Yes,2014
1785,"Hunt, H. Guy",Alabama,Montgomery,54.0,Male,White,Official Misconduct,Probation,Not Provided,0.0,...,No,No,No,No,No,No,No,No,No,Not Provided
1320,"Geeslin, Mark",Alabama,Madison,23.0,Male,White,Sexual Assault,Life,Life,2.9,...,No,No,No,No,No,No,No,No,No,Not Provided
1736,"Holsomback, John",Alabama,Shelby,33.0,Male,White,Child Sex Abuse,Term of Years,25 years,10.1,...,No,No,No,No,No,No,No,No,No,Not Provided
1723,"Holemon, Jeffrey",Alabama,Tuscaloosa,23.0,Male,White,Sexual Assault,Life,Not Provided,10.6,...,No,No,No,No,No,No,No,No,No,Not Provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266,"Sanders, Derrick",Wisconsin,Milwaukee,21.0,Male,Black,Murder,Term of Years,21 to life,25.0,...,No,No,No,No,No,No,No,No,Yes,1996
4077,"Willoughby, Troy",Wyoming,Sublette,20.0,Male,White,Murder,Life,Not Provided,2.0,...,No,No,No,No,No,No,Yes,No,No,Not Provided
1893,"Johnson, Andrew",Wyoming,Laramie,39.0,Male,Black,Sexual Assault,Life,Not Provided,23.6,...,No,No,No,No,No,No,No,No,No,Not Provided
1029,"Drennen, Gabriel",Wyoming,Fremont,36.0,Male,White,Murder,Life,Not Provided,2.9,...,No,Yes,Yes,No,No,No,Yes,No,No,Not Provided


In [9]:
# Top 5 States with the most exonerations
top5_states = exonerations_df['State'].value_counts().head()

In [10]:
top5_states

State
Illinois      608
Texas         515
New York      428
California    336
Michigan      213
Name: count, dtype: int64

In [11]:
# Top 5 Crimes with the most exonerations
top5_crime = exonerations_df['Crime'].value_counts().head()

In [12]:
top5_crime

Crime
Murder                     1698
Drug Possession or Sale     645
Sexual Assault              414
Child Sex Abuse             345
Robbery                     257
Name: count, dtype: int64

In [13]:
# Top 5 Races with the most exonerations
top5_race = exonerations_df['Race'].value_counts().head()

In [14]:
top5_race

Race
Black       2114
White       1411
Hispanic     499
Unknown       75
Asian         34
Name: count, dtype: int64

In [15]:
# What has been the highest amount of "Years lost" 
high_yearslost = exonerations_df['Years Lost'].max()

In [16]:
high_yearslost

48.1

In [17]:
# The average amount of "Years lost"
avg_yearslost = exonerations_df['Years Lost'].mean()

In [18]:
avg_yearslost

np.float64(8.754936648338512)

In [19]:
# What are the most common Ages getting convicted and exonerated
top5_ages = exonerations_df['Age at Crime Yrs'].value_counts().head()

In [20]:
top5_ages

Age at Crime Yrs
20.0    244
19.0    226
21.0    214
18.0    208
22.0    204
Name: count, dtype: int64

In [24]:
# How many men or women have been exonerated
gender_df = exonerations_df['Sex'].value_counts()

In [25]:
gender_df

Sex
Male      3849
Female     334
Name: count, dtype: int64

In [39]:
# New df showing Exonerations with Murder Charge and Inadequate Legal Defense
exonerations_df[(exonerations_df['Inadequate Legal Defense'] == "Yes") & (exonerations_df['Crime'] == "Murder")]

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
2,"Abbott, Stearns Kendall",Massachusetts,Middlesex,40.0,Male,White,Murder,Death,Not Provided,30.5,...,No,No,No,No,No,No,No,No,No,Not Provided
6,"Abramowski, Jeffrey",Florida,Brevard,40.0,Male,White,Murder,Life,Life,18.8,...,No,No,No,No,No,No,No,No,No,Not Provided
19,"Adams, Laurence",Massachusetts,Suffolk,19.0,Male,Black,Murder,Death,Not Provided,30.1,...,No,Yes,Yes,No,No,No,Yes,Yes,Yes,2003
26,"Addison, Ronald",Maryland,Baltimore City,19.0,Male,Black,Murder,Term of Years,30 years,7.7,...,No,No,No,No,No,No,Yes,No,Yes,2003
29,"Aguirre-Jarquin, Clemente",Florida,Seminole,24.0,Male,Hispanic,Murder,Death,Not Provided,12.7,...,No,No,No,No,No,No,No,No,Yes,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4141,"Wright, David",Illinois,Cook,17.0,Male,Black,Murder,Life without parole,Not Provided,25.9,...,Yes,Yes,Yes,Yes,Yes,No,No,No,Yes,2017
4154,"Yanez, Jose",Illinois,Rock Island,28.0,Male,Hispanic,Murder,Term of Years,15 years,2.4,...,No,No,No,No,No,No,No,No,No,Not Provided
4156,"Yarbough, Anthony",New York,Kings,18.0,Male,Black,Murder,Term of Years,75 to life,20.0,...,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,2005
4157,"Yarris, Nicholas",Pennsylvania,Delaware,20.0,Male,White,Murder,Death,Not Provided,21.2,...,No,Yes,Yes,No,Yes,No,Yes,Yes,Yes,1983


In [40]:
exonerations_df.columns

Index(['Name', 'State', 'County', 'Age at Crime Yrs', 'Sex', 'Race', 'Crime',
       'Sentence', 'Term of Years', 'Years Lost', 'Pre-1989', 'Date of Exon',
       'Date of 1st Convic', 'Date of Crime Month', 'Date of Crime Day',
       'Date of Crime Year ', 'Date of Release', 'Posting Date',
       'Case Characteristics', 'DNA', 'False Confession ',
       'False or Misleading Forensic Evidence?', 'Inadequate Legal Defense',
       'Mistaken Witness ID', 'Official Misconduct',
       'Perjury or False Accusation?', 'Child Welfare Worker Misconduct',
       'Exculpatory Evidence Withheld', 'Forensic Analyst Misconduct',
       'Knowingly Permitting Perjury', 'Misconduct in Interrogation',
       'Misconduct in Interrogation.1', 'Misconduct that is not withholding',
       'OM - Perjury', 'Police Misconduct', 'Prosecutor Lied in Court',
       'Prosecutor Misconduct', 'Witness Tampering', 'Recantation',
       'Year of recantation'],
      dtype='object')

In [46]:
# Trying to see which factors are more common when it comes to exonerations
factors_df = exonerations_df[['DNA', 'False Confession ',
       'False or Misleading Forensic Evidence?', 'Inadequate Legal Defense',
       'Mistaken Witness ID', 'Official Misconduct',
       'Perjury or False Accusation?', 'Child Welfare Worker Misconduct',
       'Exculpatory Evidence Withheld', 'Forensic Analyst Misconduct',
       'Knowingly Permitting Perjury', 'Misconduct in Interrogation',
       'Misconduct in Interrogation.1', 'Misconduct that is not withholding',
       'OM - Perjury', 'Police Misconduct', 'Prosecutor Lied in Court',
       'Prosecutor Misconduct', 'Witness Tampering']].apply(pd.Series.value_counts)

In [47]:
factors_df
# Seems like Perjury or False Accusation has the most "Yes"

,DNA,False Confession,False or Misleading Forensic Evidence?,Inadequate Legal Defense,Mistaken Witness ID,Official Misconduct,Perjury or False Accusation?,Child Welfare Worker Misconduct,Exculpatory Evidence Withheld,Forensic Analyst Misconduct,Knowingly Permitting Perjury,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering
No,3554,3648,3074,3112,2982,1775,1585,4130,2273,4079,3864,3925,2304,2304,3534,2543,3973,2983,3316
Yes,629,535,1109,1071,1201,2408,2598,53,1910,104,319,258,1879,1879,649,1640,210,1200,867


In [51]:
yescounts_df = factors_df.loc["Yes"]

In [52]:
yescounts_df

DNA                                        629
False Confession                           535
False or Misleading Forensic Evidence?    1109
Inadequate Legal Defense                  1071
Mistaken Witness ID                       1201
Official Misconduct                       2408
Perjury or False Accusation?              2598
Child Welfare Worker Misconduct             53
Exculpatory Evidence Withheld             1910
Forensic Analyst Misconduct                104
Knowingly Permitting Perjury               319
Misconduct in Interrogation                258
Misconduct in Interrogation.1             1879
Misconduct that is not withholding        1879
OM - Perjury                               649
Police Misconduct                         1640
Prosecutor Lied in Court                   210
Prosecutor Misconduct                     1200
Witness Tampering                          867
Name: Yes, dtype: int64